In [5]:
# imports

import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


## hyper-parameters

In [7]:
random_seed = 42

## Preprocessing and Loading dataset

In [4]:
with open('dataset.pkl', 'rb') as f:
    data = pickle.load(f)

df_uwb_data = data['data_CIR']
df_uwb = data['data_t']

In [16]:
## Train & valdiation & Test split
X_train, X_test, y_train, y_test = train_test_split(df_uwb_data.values, df_uwb['NLOS'].values, test_size=0.1, random_state=random_seed, stratify=df_uwb['NLOS'].values)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=random_seed, stratify=y_train)

print('X_train_shape {}, y_train_shape {} : '.format(X_train.shape, y_train.shape))
print('X_val_shape {}, y_val_shape {} : '.format(X_val.shape, y_val.shape))
print('X_test_shape {}, y_test_shape {} : '.format(X_test.shape, y_test.shape))

print("Train NLOS 0 count :", len(y_train[y_train==0])) 
print("Train NLOS 1 count :", len(y_train[y_train==1])) 
print("Validation NLOS 0 count :", len(y_val[y_val==0])) 
print("Validation NLOS 1 count :", len(y_val[y_val==1])) 
print("Test NLOS 0 count :", len(y_test[y_test==0])) 
print("Test NLOS 0 count :", len(y_test[y_test==1]))


X_train_shape (34020, 1016), y_train_shape (34020,) : 
X_val_shape (3780, 1016), y_val_shape (3780,) : 
X_test_shape (4200, 1016), y_test_shape (4200,) : 
Train NLOS 0 count : 17010
Train NLOS 1 count : 17010
Validation NLOS 0 count : 1890
Validation NLOS 1 count : 1890
Test NLOS 0 count : 2100
Test NLOS 0 count : 2100


In [34]:
y_train_lst = y_train.tolist()
idx_los = np.where(y_train==0)[0]
idx_nlos = np.where(y_train==1)[0]
print(idx_los)
print(idx_nlos)

[    1     2     3 ... 34015 34016 34017]
[    0     4     5 ... 34012 34018 34019]


In [17]:
class SiameseNetworkDataset():
    
    def __init__(self,training_csv=None,training_dir=None,transform=None):
        # used to prepare the labels and images path
        self.training_df=pd.read_csv(training_csv)
        self.training_df.columns =["image1","image2","label"]
        self.training_dir = training_dir    
        self.transform = transform

    def __getitem__(self,index):
        
        # getting the image path
        image1_path=os.path.join(self.training_dir,self.training_df.iat[index,0])
        image2_path=os.path.join(self.training_dir,self.training_df.iat[index,1])
        
        
        # Loading the image
        img0 = Image.open(image1_path)
        img1 = Image.open(image2_path)
        img0 = img0.convert("L")
        img1 = img1.convert("L")
        
        # Apply image transformations
        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1 , torch.from_numpy(np.array([int(self.training_df.iat[index,2])],dtype=np.float32))
    
    def __len__(self):
        return len(self.training_df)